In [ ]:
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision


In [1]:
import sys
print(sys.executable)

/Users/sarthakjain/anaconda3/envs/ml_env/bin/python


In [ ]:
model_path = 'pose_landmarker_lite.task'

In [13]:
import os
import numpy as np
import cv2

In [ ]:
import os
import numpy as np
import mediapipe as mp

BaseOptions = mp.tasks.BaseOptions
PoseLandmarker = mp.tasks.vision.PoseLandmarker
PoseLandmarkerOptions = mp.tasks.vision.PoseLandmarkerOptions
VisionRunningMode = mp.tasks.vision.RunningMode

options = PoseLandmarkerOptions(
    base_options=BaseOptions(model_asset_path=model_path),
    running_mode=VisionRunningMode.IMAGE
)

dataset_path = "/content/TRAIN"
X = []
y = []

with PoseLandmarker.create_from_options(options) as landmarker:
    for label_folder in os.listdir(dataset_path):

        label_path = os.path.join(dataset_path, label_folder)
        if not os.path.isdir(label_path):
            continue

        images_folder = os.path.join(label_path, "Images")
        if not os.path.isdir(images_folder):
            continue

        for image_file in os.listdir(images_folder):
            if not image_file.lower().endswith(('.png', '.jpg', '.jpeg')):
                continue

            full_image_path = os.path.join(images_folder, image_file)

            test_img = cv2.imread(full_image_path)
            if test_img is None:
                print(f"Skipping invalid image: {full_image_path}")
                continue  # skip to next image


            image_rgb = cv2.cvtColor(test_img, cv2.COLOR_BGR2RGB)

            # Convert to MediaPipe Image object
            mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=image_rgb)

            result = landmarker.detect(mp_image)

            if result.pose_landmarks and len(result.pose_landmarks) > 0:
                landmark_list = result.pose_landmarks[0]  # 1 person
                landmarks = []
                for lm in landmark_list:
                    landmarks.extend([lm.x, lm.y, lm.z, lm.visibility])
                X.append(landmarks)
                y.append(label_folder)

# Convert to arrays
X = np.array(X)
y = np.array(y)

print("Feature shape:", X.shape)
print("Labels shape:", y.shape)




In [17]:
print(X[0])

[ 0.8757658   0.53908175 -0.67610019  0.99999869  0.89034152  0.55701411
 -0.64062256  0.99999416  0.8876375   0.56449693 -0.64058036  0.99999738
  0.88498503  0.57207268 -0.64052153  0.99999297  0.89694303  0.53808522
 -0.64274484  0.99999595  0.89873356  0.53240204 -0.6427213   0.99999654
  0.90026587  0.52710998 -0.642658    0.99999344  0.87312973  0.57837862
 -0.42572474  0.99998832  0.89264321  0.5186516  -0.42588976  0.99999654
  0.85428524  0.54616767 -0.59736389  0.99997377  0.86434942  0.51980579
 -0.59826982  0.99998295  0.77098054  0.59526509 -0.24354801  0.99991822
  0.82456315  0.42983502 -0.31969675  0.9999764   0.74479109  0.74048346
 -0.29503557  0.98926896  0.81868935  0.27991378 -0.32880712  0.99010694
  0.75473166  0.86369467 -0.51292527  0.98435348  0.82677448  0.1524585
 -0.54001486  0.99009752  0.75565398  0.89619273 -0.55652636  0.95910573
  0.83813274  0.10977322 -0.59231228  0.97423685  0.75702089  0.8965925
 -0.6087665   0.95910197  0.83824384  0.1109426  -0.6

In [1]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)  # e.g., 'Tadasana' -> 0, 'Vrikshasana' -> 1


NameError: name 'y' is not defined

In [ ]:
for i, pose_name in enumerate(label_encoder.classes_):
    print(f"Encoded value {i} corresponds to: {pose_name}")

In [20]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded)


In [21]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential([
    Dense(128, activation='relu', input_shape=(132,)),
    Dense(64, activation='relu'),
    Dense(len(np.unique(y_encoded)), activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=20)


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 2s 29ms/step - accuracy: 0.2417 - loss: 1.9928 - val_accuracy: 0.5361 - val_loss: 1.7593
Epoch 2/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.5412 - loss: 1.6692 - val_accuracy: 0.6907 - val_loss: 1.5260
Epoch 3/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.7072 - loss: 1.4012 - val_accuracy: 0.6701 - val_loss: 1.3271
Epoch 4/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7005 - loss: 1.2694 - val_accuracy: 0.6907 - val_loss: 1.1664
Epoch 5/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7088 - loss: 1.1014 - val_accuracy: 0.8351 - val_loss: 1.0038
Epoch 6/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.8243 - loss: 0.9177 - val_accuracy: 0.8351 - val_loss: 0.8718
Epoch 7/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.8049 - loss: 0.8696 - val_accuracy: 0.7938 - val_loss: 0.8028
Epoch 8/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7641 - loss: 0.8132 - val_accuracy: 0.8969 - val_l

In [23]:


# For neural net
loss, acc = model.evaluate(X_test, y_test)
print("Validation accuracy:", acc)


4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.9471 - loss: 0.2546 
Validation accuracy: 0.9484536051750183


In [25]:
model.save("yoga_pose_model.keras")
#Saving the final trained model